# Merge Sorbetto and serve as pmtiles

It's all in the title :)
Pre-requisite: 
* Sorbetto files downloaded in the working directory below (links in https://tartamillo.wordpress.com/sorbetto/)
* s3 object store setup done in aws-cli (for the very end)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
NBROOT = os.path.realpath(os.curdir)
sys.path.append(NBROOT + '/../../eslope/development/src')
import mbt_util as M

In [4]:
%cd ~/Downloads/dwnmaps/mapdata/mbtiles/Sorbetto/

/home/eoubrayrie/Downloads/dwnmaps/mapdata/mbtiles/Sorbetto


In [4]:
import glob

paths = glob.glob('S2201/Sorbetto 2201 *.mbtiles')
assert len(paths) == 16, 'Missing paths in' + str(paths)

In [21]:
desc = '''Sorbetto 2201 : 
<p>Questa mappa è pensata per l’uso sci alpinistico in Piemonte, Val d’Aosta e zone confinanti. Lo scopo principale è conoscere a priori l’inclinazione del terreno.</p>

<p><strong>La mappa è stata elaborata con strumenti automatici. Sono state fatte verifiche a campione e apparentemente l’indicazione è abbastanza precisa, ma non posso assumermi nessuna responsabilità sull’esattezza di quanto rappresentato. I dati presenti in OpenStreetMap sono inseriti da volontari e possono risultare talvolta errati e/o incompleti. L’utilizzo è a vostro rischio e pericolo.</strong></p>

<p>L’inclinazione, calcolata con GDALDEM a partire dai DTM di Regione Piemonte, Valle d’Aosta, Liguria, Swisstopo e IGN France è rappresentata secondo il seguente schema al massimo livello di zoom disponibile:</p>

<ul>
<li>fino a 5° verde</li>
<li>da 5° a 10° eliminazione progressiva della colorazione</li>
<li>da 30° a 45° passaggio progressivo da giallo a rosso</li>
<li>da 45° a 50° colorazione rossa uniforme</li>
...
<p>La mappa è rilasciata sotto <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.it">licenza Creative Commons CC BY-NC-SA 4.0 IT</a> </p>

<p>Potete verificare l’eventuale esistenza di una versione aggiornata all’URL:<a href="https://tartamillo.wordpress.com/sorbetto/">https://tartamillo.wordpress.com/sorbetto/</a></p>
'''
M.mbt_merge(*paths,
            dest='Sorbetto.mbtiles',
            description=desc)

In [7]:
# Don't keep duplicate indexes
import sqlite3
db = sqlite3.connect('Sorbetto.mbtiles')
try:
    print(list(db.cursor().execute("SELECT name, tbl_name FROM sqlite_master WHERE type = 'index';")))
    db.cursor().execute('DROP INDEX IF EXISTS tiles_id')
    db.cursor().execute('DROP INDEX IF EXISTS tiles_idx')
finally:
    db.commit()
    db.close()


In [17]:
M.set_real_bounds('Sorbetto.mbtiles')
M.mbt_info('Sorbetto.mbtiles')

the bounds: LngLatBbox(west=6.591796874999988, south=44.02442151965934, east=8.701171875000016, north=46.49839225859762)
Falling back to first tile  12 2123 2619 LngLatBbox(west=6.591796874999988, south=44.77793589631623, east=6.679687499999988, north=44.840290651397986)


'zoom = 12 16 ; n = 157542 ; bounds = 6.5918,44.02442,8.70117,46.49839 ; center = 6.63574,44.80911,12 ; format = jpg'

In [22]:
!command -v pmtiles-convert &> /dev/null || pip install pmtiles

In [18]:
#sys.path.insert(0, NBROOT + '/../../../UPSTREAM/pmtiles/python/')
from pmtiles import convert
convert.mbtiles_to_pmtiles('Sorbetto.mbtiles', 'Sorbetto.pmtiles', None)
#!pmtiles-convert Sorbetto.{mb,pm}tiles

# of addressed tiles: 157542
# of tile entries (after RLE): 157126
# of tile contents: 156953
Root dir bytes: 196
Leaves dir bytes: 355820
Num leaf dirs: 39
Total dir bytes: 356016
Average leaf dir bytes: 9123.589743589744
Average bytes per addressed tile: 2.2598164299044066


In [11]:
# configure your s3
# pip install awscli-plugin-endpoint
# aws configure set plugins.endpoint awscli_plugin_endpoint
# aws configure

In [5]:
!aws s3 cp --acl public-read ./Sorbetto.pmtiles s3://maki/

^C


In [3]:
!aws s3api put-bucket-cors --bucket maki --cors-configuration \
    '{"CORSRules": [{"AllowedOrigins": ["*"], "AllowedHeaders": ["*"], "AllowedMethods": ["GET", "HEAD"], "MaxAgeSeconds": 3000, "ExposeHeaders": ["Etag"]}]}'

# Same, with Bugianen

In [20]:
%cd ~/Downloads/dwnmaps/mapdata/mbtiles/Bugianen/

/home/eoubrayrie/Downloads/dwnmaps/mapdata/mbtiles/Bugianen


In [2]:
import glob

paths = glob.glob('Bugianen 2205 *.mbtiles')
assert len(paths) == 12, 'Missing paths in' + str(paths)

In [5]:
M.mbt_merge(*paths,
            dest='Bugianen.mbtiles',
            description='https://tartamillo.wordpress.com/bugianen')

cp Bugianen 2205 Gran Paradiso.mbtiles Bugianen.mbtiles
<<>> Bugianen 2205 Gran Paradiso : zoom = 12 16 ; n = 19411 ; bounds = 6.943,45.398,7.910,45.768 ; format = jpg
Deduplicating Bugianen.mbtiles....
<< Bugianen 2205 Germanasca : zoom = 12 16 ; n = 15004 ; bounds = 6.592,44.778,7.471,45.089 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 34415 ; bounds = 6.59200,44.77800,7.91000,45.76800 ; format = jpg
<< Bugianen 2205 Cuneese : zoom = 12 16 ; n = 22847 ; bounds = 6.768,44.088,7.646,44.590 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 57262 ; bounds = 6.59200,44.08800,7.91000,45.76800 ; format = jpg
<< Bugianen 2205 Monviso : zoom = 12 16 ; n = 12617 ; bounds = 6.855,44.528,7.559,44.840 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 65105 ; bounds = 6.59200,44.08800,7.91000,45.76800 ; format = jpg
<< Bugianen 2205 Monregalese : zoom = 12 16 ; n = 14322 ; bounds = 7.559,44.024,8.174,44.402 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 78063 ; bounds = 6.59200,44.02400,8.17400,45

In [9]:
M.update_mbt_meta('Bugianen.mbtiles', desc=
'''Bugianen 2205 Cervino : Mappa escursionistica delle alpi piemontesi, valdostane e dintorni.
Licenza Creative Commons CC BY-NC-SA 3.0 IT.
Per la legenda e l'origine dei dati visita <a href="https://tartamillo.wordpress.com/sorbetto/">tartamillo.wordpress.com/sorbetto/</a>''')

In [13]:
# Don't keep duplicate indexes
import sqlite3
db = sqlite3.connect('Bugianen.mbtiles')
try:
    print(list(db.cursor().execute("SELECT name, tbl_name FROM sqlite_master WHERE type = 'index';")))
    db.cursor().execute('DROP INDEX IF EXISTS tiles_id')
    db.cursor().execute('DROP INDEX IF EXISTS tiles_idx')
finally:
    db.commit()
    db.close()


[('tiles_idx', 'tiles'), ('metadata_idx', 'metadata'), ('zxy', 'tiles'), ('meta', 'metadata')]


In [21]:
M.set_real_bounds('Bugianen.mbtiles')
M.mbt_info('Bugianen.mbtiles')

the bounds: LngLatBbox(west=6.591796874999988, south=44.02442151965934, east=8.789062500000012, north=46.49839225859762)
Falling back to first tile  12 2123 2620 LngLatBbox(west=6.591796874999988, south=44.840290651397986, east=6.679687499999988, north=44.902577996288855)


'zoom = 12 16 ; n = 152742 ; bounds = 6.5918,44.02442,8.78906,46.49839 ; center = 6.63574,44.87143,12 ; format = jpg'

In [22]:
from pmtiles import convert
convert.mbtiles_to_pmtiles('Bugianen.mbtiles', 'Bugianen.pmtiles', None)

# of addressed tiles: 152742
# of tile entries (after RLE): 152173
# of tile contents: 151957
Root dir bytes: 192
Leaves dir bytes: 340058
Num leaf dirs: 38
Total dir bytes: 340250
Average leaf dir bytes: 8948.894736842105
Average bytes per addressed tile: 2.2276125754540335
